In [ ]:
#Просмотр дата-сета перед загрузкой, смотрим наполненность
try:
    df = pd.read_csv('C:/Users/Вика/Downloads/prepared_data.csv', encoding='utf-8')
    print(f"Успешно загружен файл. Найдено {len(df)} записей.")
    
    # Предпросмотр данных
    print("\nПервые 5 строк данных:")
    print(df.head())
    
except Exception as e:
    print(f"Ошибка при чтении файла: {e}")
    exit()

In [ ]:
def import_to_postgres(dataframe):
    #Функция для импорта данных в PostgreSQL
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                # 1. Вставка товаров (products)
                products = dataframe[['name', 'description', 'has_sizes', 'color']].drop_duplicates()
                products['has_sizes'] = products['has_sizes'].astype(str).str.lower() == 'true'
                
                print("\nИмпорт товаров...")
                execute_batch(cur,
                    """
                    INSERT INTO products (name, description, has_sizes, color)
                    VALUES (%(name)s, %(description)s, %(has_sizes)s, %(color)s)
                    RETURNING product_id, name, color
                    """,
                    products.to_dict('records'),
                    page_size=100
                             )
                #product_map = {(row[1], row[2]): row[0] for row in cur.fetchall()}
                product_map = products.reset_index(drop=True).reset_index()
                product_map['index'] = product_map['index'] + 1
                #print(product_map.head())
                print(f"Добавлено {product_map.shape[0]} товаров")
                
                # 2. Вставка пользователей (users)
                users = dataframe[['reviewerName']].drop_duplicates()
                
                print("\nИмпорт пользователей...")
                execute_batch(cur,
                    """
                    INSERT INTO users (reviewer_name)
                    VALUES (%(reviewerName)s)
                    RETURNING user_id, reviewer_name
                    """,
                    users.to_dict('records'),
                    page_size=100
                )
                #user_map = {row[1]: row[0] for row in cur.fetchall()}
                user_map = users.reset_index(drop=True).reset_index()
                user_map['index'] = user_map['index'] + 1
                print(f"Добавлено {user_map.shape[0]} пользователей")
                
                # 3. Вставка отзывов (reviews)
                print("\nИмпорт отзывов...")
                batch_size = 500
                total_reviews = 0
                
                # Разбиваем на батчи для прогресс-бара
                for i in tqdm(range(0, len(dataframe), batch_size), desc="Обработка отзывов"):
                    batch = dataframe.iloc[i:i+batch_size]
                    reviews_data = []
                    
                    for _, row in batch.iterrows():
                        product_key = (row['name'], row['color'])
                              
                        #if product_key in product_map and row['reviewerName'] in user_map:
                        df_product_tmp = product_map[(product_map['name']==row['name']) & (product_map['color']==row['color'])].reset_index()
                        product_id = df_product_tmp['index'][0]
                        df_user_tmp = user_map[user_map['reviewerName']==row['reviewerName']].reset_index()
                        user_id = df_user_tmp['index'][0]
                         
                        try:
                            reviews_data.append({
                                'product_id': int(product_id), #product_map[product_key],
                                'user_id': int(user_id), #user_map[row['reviewerName']],
                                'text': str(row['text']),
                                'is_obscene': str(row['isObscene']).lower() == 'true',
                                'matching_size': row['matchingSize'] if pd.notna(row['matchingSize']) else None,
                                'mark': int(row['mark']) if pd.notna(row['mark']) else None
                            })
                        except Exception as e:
                            print (e)
                    
                    # Фильтруем отзывы с корректными оценками - дополнительная проверка перед загрузкой в бд
                    valid_reviews = [r for r in reviews_data if r['mark'] and 1 <= r['mark'] <= 5]
                    
                    if valid_reviews:
                        execute_batch(cur,
                            """
                            INSERT INTO reviews (product_id, user_id, text, is_obscene, matching_size, mark)
                            VALUES (%(product_id)s, %(user_id)s, %(text)s, %(is_obscene)s, %(matching_size)s, %(mark)s)
                            """,
                            valid_reviews,
                            page_size=100
                        )
                        total_reviews += len(valid_reviews)
                
                conn.commit()
                print(f"\nИмпорт завершен! Добавлено {total_reviews} отзывов")
                return True
    
    except Exception as e:
        print(f"\nОшибка при импорте в БД: {e}")
        return False

# Запуск импорта
if __name__ == "__main__":
    print("\nНачало импорта в PostgreSQL...")
    if import_to_postgres(df):
        print("Импорт успешно завершен!")
    else:
        print("Импорт завершен с ошибками")